In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
import loadexcel as lc
from tools import filetool as ft
import loaddb as ld
import datetime
pd.options.display.max_rows = 20

In [23]:
date=datetime.datetime(2019,8,1)
cleaned=lc.excel2Dataframe(path="example_data//feedback//8月原始",target_by="车号",date_=date)

['12']
['12']
['50']
['50']
['156']
['156']
['98']
['98']
['15']
['15']
['23']
['23']
['88']
['88']
['63']
['63']
['71']
['71']
['81']
['81']
['60']
['60']
['193']
['193']
['101', '2']
['101', '2']
['101']
['101']
['1']
['1']
['17']
['17']
['17', '2']
['17', '2']
['28']
['28']
['501']
['501']
['161']
['161']
['22']
['22']
['106']
['106']
['108']
['108']
['200']
['200']
['105']
['105']
['107']
['107']
['8']
['8']
['199']
['199']
['35']
['35']
['30']
['30']
['31']
['31']
['149']
['149']
['21']
['21']
['83']
['83']
['83']
['83']
['155']
['155']
['67']
['67']
['57']
['57']
['90']
['90']
['103']
['103']
['136']
['136']
['7']
['7']
['14']
['14']
['112']
['112']
['113']
['113']
['115']
['115']
['162']
['162']
['502']
['502']


In [24]:
sum_,detail_=[],[]
for item in cleaned:
    sum_+=item[0]
    detail_+=item[1]

In [25]:
data=pd.concat(detail_)

In [26]:
remove_route=lambda x:x.split("路")[-1].strip()
remove_line=lambda x:x.split("线")[-1].strip()
remove_diagonal=lambda x:x.split("/")[-1].strip()

In [27]:
#将所选列转为字符
data.iloc[:,0]=data.iloc[:,0].apply(str).apply(remove_route).apply(remove_line).apply(remove_diagonal)

In [28]:
car_value=ld.get_car_info()

In [29]:
data.index=range(len(data))

In [30]:
data.iloc[:,0]=data.iloc[:,0].apply(str)
data.iloc[:,0]=data.iloc[:,0].str.strip()

In [31]:
#将F改成正常拼写
replaceF=lambda x:x.replace("Ｆ","F")
#如果以F和D结尾，则补齐6位
zfill_ = lambda x: x.zfill(6) if x.endswith("F")|x.endswith("D") else x
#增加F末尾并补齐
zfill_F = lambda x: "".join([x.zfill(5),"F"])
#增加D末尾并补齐
zfill_D = lambda x: "".join([x.zfill(5),"D"])
#增加A开头
zfill_a=lambda x: "".join(["A",x])
zfill_b=lambda x: "".join(["B",x])
#去掉空格
replaceBk=lambda x:x.replace(" ","")

In [32]:
data["car_id"]=data.iloc[:,0].apply(replaceF)
data["car_id"]=data.iloc[:,0].apply(zfill_)
#data["car_id"]=data.iloc[:,0].apply(replaceBk)

In [39]:
#连接油耗数据与指标
result_data=pd.merge(data,car_value,left_on='car_id',right_on='sub_car_id',how='left')

In [40]:
outer_result=result_data[pd.isnull(result_data['sub_car_id'])]

In [41]:
outer_result

,car_id_x,fix_days,stop_days,work_days,engage_mileage,public_mileage,shunt_mileage,fault_times,fault_minutes,route_x,...,sub_car_id,cartype_id,car_id_y,route_y,id,target_value1,target_value2,target_value3,target_value4,power_type
392,600,0,0,31,0,0,390.8,0,0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
un_turn_data=outer_result[["route_x","team","car_id_x"]]

In [37]:
for i in range(len(un_turn_data)) :
    _car=un_turn_data.iloc[i]
    car_id_=_car["car_id_x"]
    route_=_car["route_x"]
    _r=car_value[car_value["route"]==route_]
    result_r=_r[_r["sub_car_id"].str.contains(car_id_)]
    if len(result_r)==1:
        un_turn_data.iloc[i]["car_id_x"]=result_r["sub_car_id"].iloc[0]

In [38]:
#esult_data[un_turn_data.index]['sub_car_id']=un_turn_data['ar_id_x']
data.iloc[un_turn_data.index,0]=un_turn_data['car_id_x']

In [ ]:
#unturn_tmp=outer_result

In [ ]:
#unturn_tmp[['car_id_x','route']].values

In [ ]:
#unturn_tmp.iloc[:,0].str.strip()

In [ ]:
#data.iloc[unturn_tmp.iloc[0].name,0]=checked_id

In [ ]:
#data.iloc[unturn_tmp.iloc[0].name]

In [ ]:
#checked_id=car_value[car_value['sub_car_id'].str.contains(_car_id)].iloc[0]['sub_car_id']

In [ ]:
#data.iloc[data_outer.index,0]=_car_id

In [ ]:
#data_outer=outer_result.iloc[:,0].apply(zfill_F)

In [ ]:
#data_outer

In [42]:
result_data=result_data.drop(outer_result.index)

In [43]:
#直接给data赋值
#data.iloc[data_outer.index,0]=data_outer

In [44]:
#data.iloc[data_outer.index,0]

In [45]:
#data

In [46]:
#result_data=pd.merge(data,car_value,left_on='car_id',right_on='sub_car_id',how='left')

In [47]:
#outer_result=result_data[pd.isnull(result_data['sub_car_id'])]

In [48]:
#outer_result

In [49]:
#unturn_tmp=unturn_tmp[outer_result.index]

In [50]:
#unturn_tmp

In [51]:
import datetime
date=datetime.datetime(2019,7,1)

In [52]:
result_data["target_oil_cost"]=result_data["target_value2" if date.month in range(6,10) else "target_value1"]
result_data["target_elc_cost"]=result_data["target_value4" if date.month in range(6,10) else "target_value3"]
result_data["total_oil_target"]=result_data["mileage"]*result_data["target_oil_cost"]/100
result_data["total_elc_target"]=result_data["mileage"]*result_data["target_elc_cost"]/100

KeyError: 'mileage'

In [ ]:
result_data

In [ ]:
result_data.index=result_data['route']

In [ ]:
sum_data=result_data.sum(level='route')

In [ ]:
sum_data.loc['501']

In [ ]:
index_route=sum_data.index

In [ ]:
#for i in index_route:
    #sum_data.loc[i][["oil_cost","mailage"]]

In [ ]:
result_data["route"]["21"]

In [ ]:
for i in index_route:
    result_data[result_data["route"]==i]

In [ ]:
import feedback_data as fd
import imp

In [ ]:
imp.reload(fd)

In [ ]:
from xlwt import Workbook
import datetime
newWb = Workbook()
date_ = datetime.date(2019, 6, 1)
ws = newWb.add_sheet("1路统计表")

fd.write_car_oil_cost_sum(ws,date_,result_data[result_data["route"]=="1"],team="1",route="1")
newWb.save("G:\\pythonproject\\bus_sys_dst\\output\\1.xls")

In [ ]:
teams=result_data["team"].drop_duplicates().values
teams

In [ ]:
date_ = datetime.date(2019, 6, 1)
for team in teams:
    newWb = Workbook()
    routes=result_data[result_data["team"]==team]["route"].drop_duplicates().values
    for route in routes:
        ws = newWb.add_sheet(str(route)+"路统计表")
        fd.write_car_oil_cost_sum(ws,date_,result_data[result_data["route"]==route],team=team,route=route)
    newWb.save("G:\\pythonproject\\bus_sys_dst\\output\\"+team+".xls")

In [ ]:
imp.reload(fd)

In [ ]:
newWb = Workbook()
date_ = datetime.date(2019, 6, 1)
ws = newWb.add_sheet("1路汇总表")

fd.write_car_oil_cost_detail(ws,date_,result_data[result_data["route"]=="1"],team="1",route="1")
newWb.save("G:\\pythonproject\\bus_sys_dst\\output\\1.xls")

In [9]:
["1","2","3"].remove("4")

ValueError: list.remove(x): x not in list